<a href="https://colab.research.google.com/github/superchampionoftheworld/Sentiment140/blob/main/classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import things


In [1]:
import os
import sys
import numpy as np
import matplotlib as plt
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D,MaxPooling1D, Input,Dropout,Flatten,Conv1D
from keras.models import Sequential
from keras import Model
import keras
import tensorflow as tf
import pandas as pd
import tensorflow_datasets as tfds
import nltk
import string

Load Data

In [2]:
data = pd.read_csv("training.1600000.processed.noemoticon.csv", encoding = "ISO-8859-1")
data.columns = ["Target","ID","Date","NO_QUERY","User","Text"]
data = data[['Text','Target']]
print(data[ data['Target'] == 0].size)
print(data[ data['Target'] == 4].size)

1599998
1600000


Function to clean a given piece of text

In [3]:
def clean(text):
    #very simple text cleaning; simply lowercases and removes punctuation
    cleaned = "".join(c for c in text if c not in string.punctuation)
    return cleaned.lower().replace("\n", " ")

In [4]:
data["Text"]=data["Text"].map(clean)

Tokenize the text data

In [10]:
embed_dim = 100
seq_length = 300
max_features = 100000
tokenizer = Tokenizer(num_words=max_features, split=' ')
tokenizer.fit_on_texts(data['Text'].values)
X = tokenizer.texts_to_sequences(data['Text'].values)
X = pad_sequences(X, padding = "post",truncating="post",maxlen = seq_length)
Y = data["Target"].values

In [6]:
Y = data["Target"].values

In [20]:
Y

array([0., 0., 0., ..., 1., 1., 1.])

In [16]:
Y = Y/4

In [17]:
np.unique(Y)

array([0., 1.])

Build the model

In [22]:
embed_dim

100

In [62]:
model = 0

In [66]:
inputs1 = Input(shape=(seq_length,))
embedding1 = Embedding(max_features, embed_dim,input_length=seq_length)(inputs1)
conv1 = Conv1D(filters=64, kernel_size=2,padding="same")(embedding1)
#maxpool1 = tf.keras.layers.AveragePooling1D(pool_size=2)(conv1)
#flat1 = Flatten()(maxpool1)

conv2 = Conv1D(filters=64, kernel_size=4,padding="same")(embedding1)
#maxpool2 = tf.keras.layers.AveragePooling1D(pool_size=2)(conv2)
#flat2 = Flatten()(maxpool2)

conv3 = Conv1D(filters=32, kernel_size=6,padding="same")(embedding1)
#maxpool3 = tf.keras.layers.AveragePooling1D(pool_size=2)(conv3)
#flat3 = Flatten()(maxpool3)

#conv4 = Conv1D(filters=8, kernel_size=8,padding="same")(embedding1)
#maxpool4 = tf.keras.layers.AveragePooling1D(pool_size=2)(conv4)
#flat4 = Flatten()(maxpool4)

print(maxpool1.shape)
#drop1 = Dropout(0.2)(conv1)
#pool1 = tf.keras.layers.GlobalAveragePooling1D()(conv1)
#pool2 = tf.keras.layers.GlobalAveragePooling1D()(conv2)
#pool3 = tf.keras.layers.GlobalAveragePooling1D()(conv3)
#pool4 = tf.keras.layers.GlobalAveragePooling1D()(conv4)
concat = tf.keras.layers.Concatenate()([conv1,conv2,conv3])
#flat1 = tf.keras.layers.GlobalMaxPooling1D()(concat)
flat1 = tf.keras.layers.Flatten()(concat)
dropout1 = Dropout(0.2)(flat1)
x = Dense(128)(dropout1)
x = Dropout(0.2)(x)
outputs = Dense(2, activation='softmax')(x)
model = Model(inputs=inputs1, outputs=outputs)
model.compile(loss = 'sparse_categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
model.summary()

(None, 150, 64)
Model: "functional_27"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_21 (InputLayer)           [(None, 300)]        0                                            
__________________________________________________________________________________________________
embedding_20 (Embedding)        (None, 300, 100)     10000000    input_21[0][0]                   
__________________________________________________________________________________________________
conv1d_79 (Conv1D)              (None, 300, 64)      12864       embedding_20[0][0]               
__________________________________________________________________________________________________
conv1d_80 (Conv1D)              (None, 300, 64)      25664       embedding_20[0][0]               
______________________________________________________________________

In [ ]:
model.

Split the data into train and test

In [30]:
# Define a size for train set
train_size = int(0.6 * len(X))
# Split dataset 
train_X = X[:train_size]
test_X = X[train_size:]
train_Y = Y[:train_size]
test_Y = Y[train_size:]

Aaaand... start training

In [67]:
batch_size = 200
model.fit(train_X, train_Y, epochs = 12, batch_size=batch_size,validation_data=(test_X,test_Y))

Epoch 1/3
4800/4800 [==============================] - 656s 137ms/step - loss: 0.2852 - accuracy: 0.8859 - val_loss: 1.0006 - val_accuracy: 0.5400
Epoch 2/3
4800/4800 [==============================] - 655s 136ms/step - loss: 0.2249 - accuracy: 0.9121 - val_loss: 1.1564 - val_accuracy: 0.4917
Epoch 3/3
4800/4800 [==============================] - 653s 136ms/step - loss: 0.1765 - accuracy: 0.9329 - val_loss: 1.2250 - val_accuracy: 0.5470


Test the model on a validation set

In [40]:
validation_size = 10000
X_validate = test_X[-validation_size:]
Y_validate = test_Y[-validation_size:]
# X_test = X_test[:-validation_size]
# Y_test = Y_test[:-validation_size]
score,acc = model.evaluate(test_X, test_Y, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

 298/3200 [=>............................] - ETA: 52s - loss: 1.3540 - accuracy: 0.3687

KeyboardInterrupt: ignored

Function to make predictions

In [41]:
def guess(twt):
    #vectorizing the tweet by the pre-fitted tokenizer instance
    twt = [twt]
    twt = tokenizer.texts_to_sequences(twt)
    #padding the tweet to have exactly the same shape as `embedding_2` input
    twt = pad_sequences(twt, maxlen=seq_length, dtype='int32', value=0)
    sentiment = model.predict(twt,batch_size=1,verbose = 2)[0]
    #print(sentiment)
    print("Confidence: " + str(sentiment[np.argmax(sentiment)]*100) + "%")
    if(np.argmax(sentiment) == 0):
        print("negative")
    elif (np.argmax(sentiment) == 1):
        print("positive")
    elif (np.argmax(sentiment) == 2):
        print("neutral")


In [61]:
guess("good")

1/1 - 0s
Confidence: 61.66844367980957%
positive


In [ ]:
model.save("model.h5")